## 順位の実装  
・アプリケーションから使用するスニペットのみ取得

##### モジュール

In [1]:
# device = "cuda:1"
device = "cpu"

In [2]:
import pickle
def read_bin(filename):
    with open(filename,'rb') as bf:
        bin_data = pickle.load(bf)
    return bin_data

def save_bin(filename,data):
    with open(filename,'wb') as bf:
        pickle.dump(data,bf)

In [3]:
from typing import List,Dict
from collections import defaultdict
import numpy
import pickle
import csv
import os

import torch
from transformers import Wav2Vec2ForPreTraining,Wav2Vec2Processor

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
# num = str(2)
# num = str(3)
num = str(4)
# num = str(1)

##### データ

In [39]:
import pandas as pd
test = pd.read_parquet('BirdModel_remove_test/data_cross-valid/test_'+num+'.parquet')#10293
test_index = list(test.index)

##### 関数

In [40]:
# ============= 2つのListのCos類似度算出関数
import torch

def cos_sim(v1, v2):
    # リストをテンソルに変換し、GPUに転送
    min_l = min(len(v1), len(v2))
    
    # v1 = torch.tensor(v1[:min_l]).float().cuda()
    # v2 = torch.tensor(v2[:min_l]).float().cuda()
    v1 = torch.tensor(v1[:min_l], device=device, dtype=torch.float32)
    v2 = torch.tensor(v2[:min_l], device=device, dtype=torch.float32)

    # Cos類似度の計算
    dot_product = torch.dot(v1, v2)
    norm_v1 = torch.norm(v1)
    norm_v2 = torch.norm(v2)
    cos_sim = dot_product / (norm_v1 * norm_v2)
    
    return cos_sim.item()  # 結果をPythonのfloat型に変換して返す

def to_katakana(text):
    # kakasiオブジェクトを作成
    kakasi_instance = kakasi()
    kakasi_instance.setMode("J", "K")  # J（漢字）をH（ひらがな）に変換
    kakasi_instance.setMode("H", "K")  # H（ひらがな）をK（カタカナ）に変換
    
    # カタカナに変換
    conv = kakasi_instance.getConverter()
    katakana_text = conv.do(text)
    return katakana_text

In [41]:
#=================================================マルチモーダル検索用関数
def concat_vecs(query,lang,Wikidata_id,inp_Svec):
    inp_Lvec = [0]*768
    inp_Gvec = [0]*64
    inp_Svec = [0]*256

    if query != None:
        if lang=="en":
            en_tokens = en_tokenizer(query, return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                en_model.eval()
                output = en_model(**en_tokens)
        else:
            query = to_katakana(query)
            ja_tokens = ja_tokenizer(query, return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                ja_model.eval()
                output = ja_model(**ja_tokens)
        inp_Lvec = output.last_hidden_state[0][0].tolist()# queryの分散表現

    if Wikidata_id != None and Wikidata_id in bid2Gvec:
        inp_Gvec = bid2Gvec[Wikidata_id]

    return inp_Lvec+inp_Gvec+inp_Svec

In [81]:
print(1)
en_concat_vecs_et=read_bin('new_data_cv'+num+'/en_concat_vecs_p.bin')
print(2)

1
2


検索内リスト化関数

In [82]:
import numpy as np
import scipy.io.wavfile as wavfile

def Vecs_tolist(vec,processor_mode):
    out=[]
    v=np.mean(vec,axis=1)
    return v.tolist()[0]

In [83]:
import csv

def multi_Search_topN(concat_vecs,input_v,filename,i):
    max_sim=dict()
    c2 = 0
        
    for bid,v in concat_vecs:
        print('\r%d : %d / %d' %(i,c2, len(concat_vecs)), end='')
        c2 =  c2 + 1
        cos_sim_value = cos_sim(input_v,v)
        
        if bid not in max_sim:
            max_sim[bid] = cos_sim_value
        else:
            if max_sim[bid] < cos_sim_value:
                max_sim[bid] = cos_sim_value
    save_bin('csv_cv'+num+'_p/'+filename+'.bin',max_sim)

In [84]:
input_vecs_list = read_bin("new_data_cv"+num+"/input_vecs_list_p.bin")

In [78]:
import os
files = os.listdir("csv_cv"+num+"_p")

In [79]:
exist_num = []
for file in files:
    exist_num.append(int(file.replace(".bin","")))

In [65]:
exist_num[0:3]

[1929, 1935, 1937]

In [66]:
not_exist_num = []
for idx in test_index:
    if idx not in exist_num:
        not_exist_num.append(idx)

In [85]:
not_exist_num

[10237]

In [86]:
for i in range(len(not_exist_num)):
    idx = not_exist_num[i]
    input_vecs = input_vecs_list[i]
    multi_Search_topN(en_concat_vecs_et,input_vecs,str(idx),i)

0 : 84379 / 84380